In [20]:
import pandas as pd

#Importo le informazioni riguardanti le qualità delle tasks 
tasks = pd.read_csv("../Dataset 2.0/Tasks.csv")
tasks = tasks.drop(0, axis = 0)
tasks = tasks.drop("Deadline", axis = 1)


#Importo i coefficienti di regressione di ogni agente
coeffs = pd.read_csv("../Coefficienti di Regressione/coeffs.csv")

decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Selezionare il Session ID ed il Round voluti
#Assicurarsi di selezionare lo stesso round in "RegressionModel.py" e in "OptimizationModel.py"
decisions = decisions[decisions["Session ID"] == "e7e52776-2750-4cde-a5da-093ccb8feaf9"]
decisions = decisions[decisions["Round"] == 1]
decisions = decisions.sort_values("Worker Agent ID")
decisions

,ID,Session ID,Round,Worker Agent ID,Worker Agent Reputation
0,1,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,1,0.5
7,15,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,2,0.5
2,5,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,3,0.5
5,11,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,4,0.5
3,7,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,5,0.5
1,3,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,6,0.5
6,13,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,7,0.5
4,9,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,8,0.5
8,17,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,9,0.5
9,19,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,10,0.5


In [22]:
from mip import *

m = Model()

#DATI

#Identifier del Round
Round = list(decisions["ID"])


#Peso dell'Identifier del round
wRound = list(coeffs.iloc[0])


#Value di ogni task
value = []
for i in tasks["Value"]:
    value.append(int(i))


#Peso del Value



#Complessità di ogni task
difficulty = []
for i in tasks["Difficulty"]:
    difficulty.append(float(i))


#Peso della Complessità



#Stamina consumata da ogni task
effort = []
for i in tasks["Effort Required"]:
    effort.append(int(i))

    

#Peso della Stamina consumata



#Identifier del Worker Agent
WAID = list(decisions["Worker Agent ID"])


#Peso dell'Identifier del WA
wWAID = list(coeffs.iloc[91])


#Reputazione del WA prima del round
Rep = list(decisions["Worker Agent Reputation"])


#Peso della reputation
wRep = list(coeffs.iloc[92])

#creo i range utili per la creazione di variabili
#(A, T) = (range(len(WAID)), range(len(value)))


#VARIABILI

#x[a, t] vale 1 se la task t è assegnata al WA a, 0 altrimenti
#x = [[m.add_var(var_type=BINARY) for t in T] for a in A]

#y(a) contiene la reputation del WA a fine round
#y = [m.add_var(var_type=CONTINUOUS) for a in A]


#VINCOLI

#Vincolo che impone che ogni task venga assegnata ad uno ed un solo WA
#for t in T:
#    m += xsum(x[a][t] for a in A) == 1


#OBIETTIVO

#Massimizzare la sommatoria delle reputation dei worker agent


#m.write("model.lp")
#m.optimize()